In [1]:
# Setting imports
import os
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import csv
from pathlib import Path
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import random
import plotly.express as px



In [2]:
path = "./csvs/full_ipo_list.csv"

# external CSS stylesheets to style DashBoard
external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

ipo_df = pd.read_csv(path, encoding = 'latin1')
symbol = ipo_df['Symbol']
ipo_df_year = ipo_df.groupby(['Year']).sum()
ipo_df_i = ipo_df.groupby(['Sector']).sum()

# Replace Small IPO Sectors with Other    
ipo_df_ind = ipo_df
ipo_df_ind.loc[ipo_df_ind['Volume'] < 200000, "Sector"] = "Other Sector"
ipo_df_ind = ipo_df_ind[ipo_df_ind['Sector'].notna()]

#Sratify Data Set by Different Metrics
ipo_df_year_count = ipo_df.groupby(['Year']).count()
ipo_df_i_count =  ipo_df.groupby(['Sector']).count()
ipo_df_industry = ipo_df.groupby(['Sector', 'Year']).mean()

# Create Dashboard

In [3]:
app = JupyterDash(__name__,
                external_stylesheets=external_stylesheets)

fig1 = px.bar(ipo_df_year, x = ipo_df_year.index, y = ipo_df_year['Volume'].values, labels = {'x': 'Year', 'y':'Total Volume ($)'})
fig2 = px.bar(ipo_df_i, x = ipo_df_i.index, y = ipo_df_i['Volume'].values, labels = {'x': 'Sector', 'y':'Total Volume ($)'})
fig3 = px.bar(ipo_df_year_count, x = ipo_df_year_count.index, y = ipo_df_year_count['Volume'].values, labels = {'x': 'Year', 'y':'Total Count of IPOs'})
fig4 = px.bar(ipo_df_i_count, x = ipo_df_i_count.index, y = ipo_df_i_count['Volume'].values, labels = {'x': 'Industry', 'y':'Total Count of IPOs'})
fig5 = px.pie(ipo_df_ind, values = 'Volume', names = "Sector")
app.layout = html.Div([
    html.H1("IPO's Data Over Time"),
    dcc.Graph(id='graph'),
    html.Label([
        "IPO Sectors",
        dcc.Dropdown(
            id='ticker-dropdown', clearable=False,
            value=ipo_df_industry.index.get_level_values(0), options=[ {'label': c, 'value': c} for c in ipo_df_industry.index.get_level_values(0)])
    ]),
    
    html.H2("IPO's Total Volume Over Time"),
    dcc.Graph(id = 'graph2', figure = fig1),
    
    html.H3("IPO's Total Volume By Sector"),
    dcc.Graph(id = 'graph3', figure = fig2),
    
    html.H4("IPO's Total Count By Year"),
    dcc.Graph(id = 'graph4', figure = fig3),
    
    html.H5("IPO's Total Count By Sector"),
    dcc.Graph(id = 'graph5', figure = fig4),
    
    html.H5("IPO's Market Share by Total Volume Over past 25 Years"),
    dcc.Graph(id = 'graph6', figure = fig5),

])
# Define callback to update graph
@app.callback(Output('graph', 'figure'),[Input("ticker-dropdown", "value")])
def update_figure(value):
    return px.bar(ipo_df_industry, x= value, y = ipo_df_industry['Volume'].values, title = "IPO Mean Volume over Years", labels = {'x' : 'Year', 'y' : 'Mean Volume ($)'})
# Run app an d display result inline in the notebook
app.run_server(mode='inline', debug = True, port = 8504)